In [1]:
#load program
from bs4 import BeautifulSoup
import urllib.request
import re
import numpy as np
import pandas as pd

html_page = urllib.request.urlopen('https://www.census.gov/programs-surveys/popest.html').read()
soup = BeautifulSoup(html_page, "lxml")

In [42]:
print(soup.prettify()[:500])

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <!--[if lt IE 9]><meta http-equiv="X-UA-Compatible" content="IE=EmulateIE8"  /><![endif]-->
  <!--[if gte IE 9]><meta http-equiv="X-UA-Compatible" content="IE=edge"> <![endif]-->
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <link href="/etc/designs/census/bootstrap.css" rel="styleshe


In [79]:
links_dic =  {}
def loadLinks_fromDic():
    file = open("my_web_links3.csv", "w")
    for link in soup.findAll('a', attrs={'href': re.compile("^http")}):
        soup_link = str(link['href'])
        soup_link = re.split("//", soup_link,1)[1] 
        index_code = hash(soup_link)
        links_dic[index_code] = soup_link
    for i in links_dic:
        file.write(links_dic[i] + "\n")
    file.flush()    
    file.close()
%timeit loadLinks_fromDic()
len(links_dic)

13.4 ms ± 1.7 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


92

In [71]:
#Sets are significantly faster when it comes to determining if an object is present in the set (as in x in s), but are slower than lists when it comes to iterating over their contents.
def loadLinks_setDt():
    file = open("my_web_links1.csv", "w")
    links_set =  set()
    for link in soup.findAll('a', attrs={'href': re.compile("^http")}):
        soup_link = str(link['href'])
        soup_link = re.split("//", soup_link,1)[1] 
        links_set.add(soup_link)
    for i in links_set:
        file.write(i + "\n")
    file.flush()    
    file.close()
%timeit loadLinks_setDt()
len(links_set)

36 ms ± 3.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


92

In [72]:
def loadLinks_byIndex():
    links_series = pd.Series()
    for link in soup.findAll('a', attrs={'href': re.compile("^http")}):
        soup_link = str(link['href'])
        #Split the string only at the first occurrence
        soup_link = re.split("//", soup_link,1)[1]
        index_code = str(hash(soup_link))
        links_series[index_code] = soup_link
    #csv files should always be opened in binary mode
    # csv line terminator: \r\n
    file = open("my_web_links2.csv", "w", newline='')
    file.write(links_series.to_csv(index=False, header=False))
    file.flush()    
    file.close()
%timeit loadLinks_byIndex()
links_series.count()

233 ms ± 10.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


92